In [2]:
import os
import csv
import json
import pandas as pd
from tqdm import tqdm
from pprint import pprint

DATA_PATH = "./data"

In [3]:
def create_data_frame(match_data):
    gameId = []
    gameDuration = []
    championsPicked = []
    goldDifference = []
    damageDifference = []
    win = []
    
    for version in match_data.keys():
        for data in match_data[version]:
            gameId.append(data["metadata"]["matchId"])

            if version[:5] >= "11.20":
                gameDuration.append(data["info"]["gameDuration"])
            else:
                gameDuration.append(data["info"]["gameDuration"] // 1000)

            championName = []
            goldEarned = []
            totalDamageDealtToChampions = []

            for participants in data["info"]["participants"]:
                championName.append(participants["championName"])
                goldEarned.append(participants["goldEarned"])
                totalDamageDealtToChampions.append(
                    participants["totalDamageDealtToChampions"]
                )

            championsPicked.append(championName)
            goldDifference.append(sum(goldEarned[:5]) - sum(goldEarned[5:]))
            damageDifference.append(
                sum(totalDamageDealtToChampions[:5])
                - sum(totalDamageDealtToChampions[5:])
            )
            win.append(data["info"]["participants"][0]["win"])

    df = pd.DataFrame(
        {
            "game_Id": gameId,
            "game_duration": gameDuration,
            "champions_picked": championsPicked,
            "gold_difference": goldDifference,
            "damage_difference": damageDifference,
            "win": win,
        }
    )
    return df

In [4]:
# with open(os.path.join(DATA_PATH, "MASTER_I_MatchData_1_500.json")) as json_data:
#     match_data = json.load(json_data)
#     df = create_data_frame(match_data)

# df

In [5]:
all_tier_df = []
for fileName in tqdm(os.listdir(DATA_PATH)):
    if "json" in fileName:
        with open(os.path.join(DATA_PATH, fileName)) as json_data:
            match_data = json.load(json_data)
            df = create_data_frame(match_data)
            all_tier_df.append(df)

100%|██████████| 25/25 [06:35<00:00, 15.84s/it]


In [6]:
df_concat = pd.concat(all_tier_df, ignore_index=True)
df_concat

,game_Id,game_duration,champions_picked,gold_difference,damage_difference,win
0,KR_5583030086,1501,"[Irelia, Ekko, Zed, Vayne, Aatrox, Sejuani, Ta...",-11660,-24603,False
1,KR_5582966326,1399,"[Renekton, Taliyah, Sylas, Draven, Nautilus, Y...",-878,-12928,False
2,KR_5582953729,1072,"[Kalista, Warwick, Qiyana, Jhin, Karma, Riven,...",-6809,-9867,False
3,KR_5580827743,1646,"[DrMundo, Talon, Zoe, Ashe, Sett, Jayce, Drave...",9020,11615,True
4,KR_5580787457,1463,"[Jax, Khazix, Katarina, Aphelios, Nami, Gragas...",-10026,-7898,False
...,...,...,...,...,...,...
411638,KR_5613046424,925,"[Irelia, Karthus, Ryze, Samira, Rell, Akshan, ...",-13666,-18843,False
411639,KR_5613052577,1471,"[Syndra, Viego, Jayce, Aphelios, Lux, Irelia, ...",-11210,-6252,False
411640,KR_5612968773,938,"[Tryndamere, Viego, Viktor, Jhin, Karma, Aksha...",-14171,-20732,False
411641,KR_5612903241,1958,"[Qiyana, Vayne, Ryze, Yuumi, LeeSin, Irelia, R...",-1899,-8456,True


In [7]:
df_concat = df_concat.drop_duplicates(subset=["game_Id"])
df_concat

,game_Id,game_duration,champions_picked,gold_difference,damage_difference,win
0,KR_5583030086,1501,"[Irelia, Ekko, Zed, Vayne, Aatrox, Sejuani, Ta...",-11660,-24603,False
1,KR_5582966326,1399,"[Renekton, Taliyah, Sylas, Draven, Nautilus, Y...",-878,-12928,False
2,KR_5582953729,1072,"[Kalista, Warwick, Qiyana, Jhin, Karma, Riven,...",-6809,-9867,False
3,KR_5580827743,1646,"[DrMundo, Talon, Zoe, Ashe, Sett, Jayce, Drave...",9020,11615,True
4,KR_5580787457,1463,"[Jax, Khazix, Katarina, Aphelios, Nami, Gragas...",-10026,-7898,False
...,...,...,...,...,...,...
411638,KR_5613046424,925,"[Irelia, Karthus, Ryze, Samira, Rell, Akshan, ...",-13666,-18843,False
411639,KR_5613052577,1471,"[Syndra, Viego, Jayce, Aphelios, Lux, Irelia, ...",-11210,-6252,False
411640,KR_5612968773,938,"[Tryndamere, Viego, Viktor, Jhin, Karma, Aksha...",-14171,-20732,False
411641,KR_5612903241,1958,"[Qiyana, Vayne, Ryze, Yuumi, LeeSin, Irelia, R...",-1899,-8456,True


In [8]:
df_concat.to_csv("Champions_picked.csv", index=False)